## 1. 정보공개청구

### 1-1. 출장기록 정리

In [ ]:
import pandas as pd
import io

# 파일을 구글 코랩에 업로드합니다.
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 엑셀 파일을 읽어옵니다.
# If the file is .xls, use 'xlrd' engine
if filename.endswith('.xls'):
    raw_data = pd.read_excel(io.BytesIO(uploaded[filename]), engine='xlrd')
# If the file is .xlsx, use 'openpyxl' engine
elif filename.endswith('.xlsx'):
    raw_data = pd.read_excel(io.BytesIO(uploaded[filename]), engine='openpyxl')
else:
    raise ValueError(f"Unsupported file format: {filename}")

# 데이터 처리를 시작합니다.
raw_data = raw_data[pd.to_numeric(raw_data.iloc[:,0], errors='coerce').notna()]
raw_data[['출발일시', '도착일시']] = raw_data['출장기간'].str.split(' ~ ', expand=True)
raw_data[['출발일', '출발시간']] = raw_data['출발일시'].str.split(pat=' ', n=1, expand=True)
raw_data[['도착일', '도착시간']] = raw_data['도착일시'].str.split(pat=' ', n=1, expand=True)
raw_data[['이름', '코드']] = raw_data['성명'].str.split(pat='(', n=1, expand=True)
raw_data['코드'] = raw_data['코드'].str.replace(')', '')

# 필요없는 열을 삭제하고 원하는 순서대로 정렬합니다.
desired_order = ["순번", "소속/직급", "성명", "이름", "코드", "출발일", "출발시간", "도착일", "도착시간",
                 "총출장시간", "구분", "공용차량", "출장지", "출장목적", "여비등급", "일비", "식비", "숙박비",
                 "교통비", "기타", "합계", "청구액\n(수령액)"]

df_reordered = raw_data[desired_order]

# 결과를 엑셀 파일로 저장하고 다운로드합니다.
output_path = 'cleaned_data_colab.xlsx'
df_reordered.to_excel(output_path, index=False)
files.download(output_path)

In [5]:
import shutil
import os

# /content 폴더 내 모든 파일과 폴더 삭제
folder_path = '/content'

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.remove(file_path)  # 파일 또는 링크 삭제
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # 폴더 삭제
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')


### 1-2. 이름 마스킹 하기

In [ ]:
# 1. openpyxl 설치 (처음 1회만)
# !pip install openpyxl

# 2. 필요한 라이브러리 불러오기
import pandas as pd
from google.colab import files

# 3. 사용자로부터 엑셀 파일 업로드 받기
uploaded = files.upload()  # 엑셀파일 선택해서 업로드

# 4. 업로드한 파일명 자동으로 가져오기
file_name = next(iter(uploaded))  # 업로드된 첫 파일의 이름

# 5. 엑셀 파일 불러오기
df = pd.read_excel(file_name)

# 6. 이름 마스킹 함수 정의
def mask_name(name):
    if isinstance(name, str) and len(name) > 1:
        return name[0] + '0' * (len(name) - 1)
    return name

# 7. '이름' 컬럼 덮어쓰기 (이름을 마스킹한 값으로 대체)
if '이름' in df.columns:
    df['이름'] = df['이름'].apply(mask_name)
    print("✅ 이름 마스킹 후 덮어쓰기 완료.")
else:
    raise ValueError("❌ 엑셀 파일에 '이름'이라는 컬럼이 존재하지 않습니다.")

# 8. 결과 미리보기
display(df[['이름']].head())

# 9. 결과 저장 및 다운로드
output_file = 'masked_name_overwritten.xlsx'
df.to_excel(output_file, index=False)
files.download(output_file)


In [ ]:
import shutil
import os

# /content 폴더 내 모든 파일과 폴더 삭제
folder_path = '/content'

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.remove(file_path)  # 파일 또는 링크 삭제
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # 폴더 삭제
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')


## 2. 실내공기질 실적 처리

### 2-1. 데이터 병합, 추출

In [ ]:
import pandas as pd

# 데이터를 읽어오기(실험노트, 조회통계 자료 excel통합문서 양식으로 저장한 다음 저장)
data_df = pd.read_excel('실험노트.xlsx')
result_df = pd.read_excel('조회통계.xlsx')

# '접수번호'를 기준으로 데이터 병합
merged_df = pd.merge(data_df, result_df, on='접수번호', how='inner')

# 병합된 데이터를 Excel 파일로 저장
merged_df.to_excel('1.병합 데이터.xlsx', index=False)

# 병합 데이터에서 '실내공기질'이 포함된 데이터만 필터링
filtered_df = merged_df[merged_df['검체유형_x'].str.contains('실내공기질', na=False)]

# 필터링된 데이터 확인
print(filtered_df.head())

# 대기 제외, 실내공기질만 필터링된 데이터를 Excel 파일로 저장
filtered_df.to_excel('2.실내공기질 필터링 데이터.xlsx', index=False)

# '시설군' 컬럼 추가 (검체유형_x 열에서 파생)
filtered_df['시설군'] = filtered_df['검체유형_x'].str.split('/').str[2]

# 지정된 열 선택(필요한 변수만 추출)
selected_columns = [
    '의뢰기관', '시설군', '배출시설', '시설명', '채취장소_x', '시료명_x',
    'PM10', 'PM2.5', 'CO2', '폼알데하이드', '부유세균', '일산화탄소', '라돈', '라돈(밀폐)', '벤젠', '톨루엔', '에틸벤젠', '자일렌', '스틸렌', '부적합항목', '확인일', '검사결과','접수번호', '접수일자'
]

# 해당 열만 포함된 새 DataFrame 생성
extracted_df = filtered_df[selected_columns]

# 필요한 경우, 데이터를 Excel 파일로 저장
extracted_df.to_excel('3.추출 데이터.xlsx', index=False)

# 데이터 확인
print(extracted_df.head())


### 2. 기존 신축 공동주택 추출

In [ ]:
import pandas as pd

# 엑셀 파일 로드
file_path = '3.추출 데이터.xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1')

# 열 이름 변경
rename_columns = {
    '의뢰기관': '시군명',
    '배출시설': '세부시설군',
    '채취장소_x': '주소',
    'PM10': '미세먼지(PM10)',
    'PM2.5': '초미세먼지(PM2.5)',
    'CO2': '이산화탄소',
    '부유세균': '총부유세균',
    '시료명_x': '시료명'
}

df.rename(columns=rename_columns, inplace=True)

# '시설군'에서 '기존공동주택'과 '신축공동주택'만 필터링
filtered_df = df[df['시설군'].isin(['기존공동주택', '신축공동주택'])]

# 열 순서 정의 및 데이터 정렬
desired_column_order = [
    '시군명', '시설군', '시설명', '세부시설군', '주소', '시료명', '검사결과', '부적합항목', '접수일자',
    '확인일', '접수번호_10자리', '접수번호', '채취지점', '미세먼지(PM10)', '초미세먼지(PM2.5)',
    '이산화탄소', '폼알데하이드', '총부유세균', '일산화탄소','라돈', '라돈(밀폐)', '벤젠', '톨루엔',
    '에틸벤젠', '자일렌', '스틸렌'
]

# 실제 데이터에 존재하는 열만 선택하도록 처리
filtered_df = filtered_df[[col for col in desired_column_order if col in filtered_df.columns]]

# 필터링된 데이터 출력 또는 저장
filtered_df.to_excel('4.기존신축공동주택 데이터.xlsx', index=False)

print(filtered_df)

### 2-3. 다중이용시설 데이터 처리

In [ ]:
import pandas as pd

# 데이터 파일 로드
file_path = '3.추출 데이터.xlsx'
data = pd.read_excel(file_path)

# 열 이름 변경
rename_columns = {
    '의뢰기관': '시군명',
    '배출시설': '세부시설군',
    '채취장소_x': '주소',
    'PM10': '미세먼지(PM10)',
    'PM2.5': '초미세먼지(PM2.5)',
    'CO2': '이산화탄소',
    '부유세균': '총부유세균'
}
data.rename(columns=rename_columns, inplace=True)

# 텍스트 데이터를 숫자로 변환 및 "불검출" 처리
columns_to_clean = ['미세먼지(PM10)', '초미세먼지(PM2.5)', '이산화탄소', '폼알데하이드', '총부유세균', '일산화탄소', '라돈', '벤젠', '톨루엔', '에틸벤젠', '자일렌', '스틸렌']
for col in columns_to_clean:
    if col in data.columns:
        data[col] = data[col].replace("불검출", None)  # "불검출"을 NaN으로 처리
        data[col] = pd.to_numeric(data[col], errors='coerce')  # 숫자로 변환 가능한 값만 남김

# 접수번호의 앞 10자리를 기준으로 새로운 열 생성
data['접수번호_10자리'] = data['접수번호'].str[:10]

# 수치 데이터의 평균 계산 (접수번호의 앞 10자리를 기준으로)
numeric_columns = ['미세먼지(PM10)', '초미세먼지(PM2.5)', '이산화탄소', '폼알데하이드', '총부유세균', '일산화탄소', '라돈']
data_avg = data.groupby(['시설명', '접수번호_10자리']).agg({col: 'mean' for col in numeric_columns if col in data.columns}).reset_index()

# 시료명 데이터 결합 및 채취지점 생성
data['채취지점'] = data.groupby('시설명')['시료명_x'].transform(lambda x: ', '.join(x.unique()))

# 채취지점 데이터와 합치기
columns_to_drop = numeric_columns + ['미세먼지(PM10_1)', '초미세먼지(PM2.5_1)', '미세먼지(PM10_2)', '초미세먼지(PM2.5_2)', 'CO2', '이산화탄소', '일산화탄소']
columns_to_drop = [col for col in columns_to_drop if col in data.columns]  # 실제 존재하는 열만 제거
data_merged = data_avg.merge(data.drop(columns=columns_to_drop).drop_duplicates(['시설명', '접수번호_10자리']), on=['시설명', '접수번호_10자리'])

# 불필요한 시료명_x 열 제거
if '시료명_x' in data_merged.columns:
    data_merged.drop(columns='시료명_x', inplace=True)

# 1. 검사결과가 '적합'이면 부적합항목을 빈칸으로 설정
data_merged.loc[data_merged['검사결과'] == '적합', '부적합항목'] = ""

# 2. '시설군'에서 '기존공동주택', '신축공동주택' 제거
data_filtered = data_merged[~data_merged['시설군'].isin(['기존공동주택', '신축공동주택'])]

# 열 순서 정의 및 데이터 정렬
desired_column_order = [
    '시군명', '시설군', '시설명', '세부시설군', '주소', '검사결과', '부적합항목', '접수일자',
    '확인일', '접수번호_10자리', '접수번호', '채취지점', '미세먼지(PM10)', '초미세먼지(PM2.5)',
    '이산화탄소', '폼알데하이드', '총부유세균', '일산화탄소', '라돈', '라돈(밀폐)', '벤젠', '톨루엔',
    '에틸벤젠', '자일렌', '스틸렌'
]

# 열 순서 변경 및 데이터 정렬
data_sorted = data_filtered[desired_column_order].sort_values(by=['시군명', '시설군', '세부시설군', '시설명', '주소', '채취지점'])

# 결과 파일로 저장 (xlsx 파일로 저장)
output_path = '5.다중이용시설 데이터.xlsx'
data_sorted.to_excel(output_path, index=False)


### 2-4. 데이터 통합

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font

# 두 개의 엑셀 파일 로드
file_1 = '4.기존신축공동주택 데이터.xlsx'
file_2 = '5.다중이용시설 데이터.xlsx'

# 각 파일의 데이터를 불러옴
df1 = pd.read_excel(file_1)
df2 = pd.read_excel(file_2)

# 1. 숫자 데이터를 올바른 형식으로 변환
def convert_numeric_with_preservation(df, columns):
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').combine_first(df[col])  # 숫자로 변환 가능한 값은 숫자로 변환, 나머지는 그대로 유지
    return df

# 변환할 열 리스트
columns_to_convert = ['벤젠', '톨루엔', '에틸벤젠', '자일렌', '스틸렌']

# 기존신축공동주택 시트 데이터에 변환 적용
df1 = convert_numeric_with_preservation(df1, columns_to_convert)

# 2. 확인일 변수를 날짜 형식으로 변환하고, 시간 제거 및 데이터프레임을 오름차순으로 정렬
def sort_by_date(df, date_column):
    # '확인일'을 날짜 형식으로 변환 (시간 제거)
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce').dt.date  # 날짜만 남기고 시간 제거
    df = df.sort_values(by=date_column, ascending=True)  # 오름차순 정렬
    return df

# 확인일 기준 정렬 적용
df1 = sort_by_date(df1, '확인일')
df2 = sort_by_date(df2, '확인일')

# 3. 다중이용시설 데이터에서 "접수번호" 변수 제거
if '접수번호' in df2.columns:
    df2 = df2.drop(columns=['접수번호'])

# 결과를 하나의 엑셀 파일로 저장 (두 개의 시트에 각각 저장)
output_file = '6.통합 데이터.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    df2.to_excel(writer, sheet_name='다중이용시설', index=False)  # 결과를 시트1에 저장
    df1.to_excel(writer, sheet_name='기존신축공동주택', index=False)  # 기존, 신축공동주택 데이터를 시트2에 저장

# 저장된 엑셀 파일을 다시 불러옴
wb = load_workbook(output_file)

# 각 시트에 대해 서식 지정
for sheet in wb.sheetnames:
    ws = wb[sheet]

    # 글자 크기를 10으로 설정
    for row in ws.iter_rows():
        for cell in row:
            cell.font = Font(size=10)

    # 모든 열의 너비를 11로 고정
    for col in ws.columns:
        column = col[0].column_letter  # 열 이름 가져오기 (A, B, C...)
        ws.column_dimensions[column].width = 11  # 열 너비 고정

# 파일 저장
wb.save(output_file)

print("엑셀 파일이 서식과 함께 '6.통합 데이터.xlsx'로 저장되었습니다.")

In [ ]:
import shutil
import os

# /content 폴더 내 모든 파일과 폴더 삭제
folder_path = '/content'

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.remove(file_path)  # 파일 또는 링크 삭제
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # 폴더 삭제
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')


## 3. 출장지 선정

### 3-1. 중복시설 제거

In [1]:
import pandas as pd

# 파일 경로 설정
file_path = '다중이용시설 실내공기질 검사 대상시설.xlsx'  # 원본 파일 경로
output_file_path = 'updated 대상시설.xlsx'  # 저장할 파일 경로

# Excel 파일 읽기
data_2025 = pd.read_excel(file_path, sheet_name='2025')
data_2024 = pd.read_excel(file_path, sheet_name='2024')

# 열 이름 공백 제거
data_2025.columns = data_2025.columns.str.strip()
data_2024.columns = data_2024.columns.str.strip()

# 비교 대상 열 설정
columns_2025 = ['시군', '시설명']
columns_2024 = ['시군', '시설명']

# 필요한 열만 필터링
data_2025_filtered = data_2025[columns_2025]
data_2024_filtered = data_2024[columns_2024]

# 2025 데이터에 "2024년 검사 완료" 여부 추가
data_2025['2024년 검사 완료'] = data_2025_filtered.apply(
    lambda row: '2024년 검사 완료' if ((data_2024_filtered['시군'] == row['시군']) &
                                      (data_2024_filtered['시설명'] == row['시설명'])).any() else '', axis=1
)

# 업데이트된 데이터를 새로운 Excel 파일로 저장
data_2025.to_excel(output_file_path, index=False)

print(f"파일이 저장되었습니다: {output_file_path}")


파일이 저장되었습니다: updated 대상시설.xlsx


In [2]:
import shutil
import os

# /content 폴더 내 모든 파일과 폴더 삭제
folder_path = '/content'

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.remove(file_path)  # 파일 또는 링크 삭제
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # 폴더 삭제
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')


## 4. 유튜브 영상 다운로드

In [ ]:
pip install yt_dlp xlsxwriter

In [ ]:
import yt_dlp
import xlsxwriter
import os

url = "https://youtu.be/cAL-Z_FfObI?si=ZxsGC75_kqkHLRPn"

# Define options to list available formats
list_opts = {
    'listformats': False,  # Do not use the built-in listformats to allow more detailed output
}

try:
    # List available formats
    with yt_dlp.YoutubeDL(list_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        formats = info.get('formats', [])
        print("Available formats:")

        # Create a workbook and add a worksheet
        workbook = xlsxwriter.Workbook('available_formats.xlsx')
        worksheet = workbook.add_worksheet()

        # Write headers
        worksheet.write('A1', 'Format ID')
        worksheet.write('B1', 'Resolution')
        worksheet.write('C1', 'Filesize')
        worksheet.write('D1', 'Note')
        worksheet.write('E1', 'Type')
        worksheet.write('F1', 'Extension')

        # Write format details to worksheet
        row = 1
        for f in formats:
            format_id = f['format_id']
            resolution = f.get('height', 'audio only') if 'height' in f else 'unknown'
            filesize = f.get('filesize', 'unknown')
            format_note = f.get('format_note', 'unknown')
            ext = f.get('ext', 'unknown')
            media_type = 'audio' if f.get('vcodec', 'none') == 'none' else 'video'

            # Print to console
            print(f"Format ID: {format_id}, Resolution: {resolution}, Filesize: {filesize}, Note: {format_note}, Type: {media_type}, Extension: {ext}")

            # Write to Excel file
            worksheet.write(row, 0, format_id)
            worksheet.write(row, 1, resolution)
            worksheet.write(row, 2, filesize)
            worksheet.write(row, 3, format_note)
            worksheet.write(row, 4, media_type)
            worksheet.write(row, 5, ext)
            row += 1

        # Close the workbook
        workbook.close()
        print("Format list saved to available_formats.xlsx")

    # Prompt the user to select a video format
    selected_video_format = input("Enter the video format ID you want to download: ")

    # Find the selected video format in the formats list
    video_format = next((f for f in formats if f['format_id'] == selected_video_format), None)
    if not video_format:
        raise ValueError("Invalid video format ID selected.")

    # Define options to download the selected video format with throttled download speed
    video_output = 'video_only.' + video_format['ext']
    ydl_opts_video = {
        'format': selected_video_format,  # Use the selected format ID for video
        'outtmpl': video_output,  # Output file name format for video
        'ratelimit': 500000,  # Limit download speed to 500 KB/s
    }

    # Download the video in the selected format
    with yt_dlp.YoutubeDL(ydl_opts_video) as ydl:
        ydl.download([url])
    print("Video downloaded successfully!")

    # Prompt the user to select an audio format
    selected_audio_format = input("Enter the audio format ID you want to download: ")

    # Find the selected audio format in the formats list
    audio_format = next((f for f in formats if f['format_id'] == selected_audio_format), None)
    if not audio_format:
        raise ValueError("Invalid audio format ID selected.")

    # Define options to download the selected audio format
    audio_output = 'audio_only.' + audio_format['ext']
    ydl_opts_audio = {
        'format': selected_audio_format,  # Use the selected format ID for audio
        'outtmpl': audio_output,  # Output file name format for audio
        'ratelimit': 500000,  # Limit download speed to 500 KB/s
    }

    # Download the audio in the selected format
    with yt_dlp.YoutubeDL(ydl_opts_audio) as ydl:
        ydl.download([url])
    print("Audio downloaded successfully!")

    # Merge video and audio using ffmpeg
    merged_output = 'final_output.mp4'
    os.system(f"ffmpeg -i {video_output} -i {audio_output} -c:v copy -c:a aac -strict experimental {merged_output}")
    print(f"Video and audio merged successfully into {merged_output}!")

except Exception as e:
    print(f"An error occurred: {e}")
